# Tutorial 2: Running the hydrodynamic model
The hydrodynamic model is a key component of the EGM Framework as all variables are direct or indirect related to water depths, velocity and/or flow. This model is quite robust and demands many input files. Indeed, most of the `.dat` files found in the simulation folder.


This tutorial will show how to run this model using some of the classes/functions found in the **egmlib** library. We will also examine the outputs and put the model to run again using the final state of the first run as the initial condition for the second run. Through that, you will learn some of the *under the hood* operations that the EGM Framework does with the input/output files of this model.


## Dependencies
* You will need all files created with `Tutorial 1 - Creating a new domain`.
* All files in the `Common Files` folder. This folder should be inside the folder with all Tutorials' files.


## STEP 0: Load the egmlib library
In the code below replace the string `lib_folder` with the path to folder where the **egmlib.py** file is and than run the cell. Just note that we should use `/` or `\\` instead of a single `\` as Python understand this last as a indicator of a special character.


The egmlib library already include the import of many other libraries, as **numpy** and **matplotlib** that will be used later in this notebook.

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'
from sys import path
path.append(lib_folder)
from egmlib import *

## STEP 1: Files checklist
As mentioned, the hydrodynamic model read multiple **\*.dat** files, and they need to be all in the same folder. For this tutorial we need to make a directory named `tutorial02` inside the folder where we are running all tutorials. The absolute path to this folder need to be provide as a string to the variable `run_folder` in the code below.


Once created the folder, copy all **.dat** files created with Tutorial 1, as well all files found in the directory **Common Files** to the *run_folder* one. Run the code below and check if you got an "okay!" for all files.

In [ ]:
run_folder = 'C:/Wetlands/Tutorials/tutorial02'
filelist = os.listdir(run_folder)

#Check list
print('\nFiles shared between EGM Framework and Hydrodynamic model:')
for nick in ['exec', 'setup', 'bounds', 'init', 'param', 'domain']:
    if fn_[nick] in filelist:
        print('> %-20s ... okay!' % fn_[nick])
    else:
        print('> %-20s ... not found!' % fn_[nick])

print('\nFiles used by the Hydrodynamic model only')
for file in ['contab.dat','gase.dat','gener.dat','h1h2.dat','hrugosi.dat','lluvia.dat']:
    if file in filelist:
        print('> %-20s ... okay!' % file)
    else:
        print('> %-20s ... not found!' % file)

print('\nFiles used by the EGM Framework only:')
for nick in ['tiles', 'profiles']:
    if fn_[nick] in filelist:
        print('> %-20s ... okay!' % fn_[nick])
    else:
        print('> %-20s ... not found!' % fn_[nick])

## STEP 2: Running the Hydrodynamic model
As all files are in one place, we are good to run the hydrodynamic model. First we need to change the current folder to the *run_folder*. Then, we can use the function **hydrodynamic_model(** *args* **)** to run the model and get its outputs after the execution.


It can take a couple of minutes to run the model. You will know if the model is running if the size of the output txt files is increasing continously

In [ ]:
# Changing current folder to that one where we have the model's executable
os.chdir(run_folder)

# Calling function to run the model and read the outputs
t0 = datetime.now()
data1 = hydrodynamic_model(execname=fn_['exec'], return_outcome=True)
print('Time elapsed =', (datetime.now() - t0))

# Examing ouput data variable
print('Data type')
display(type(data1).__name__)
print('Data attributes')
display(dir(data1))
print('Number of time steps:', data1.NoTS)
print('Timestep (s) between records:', data1.Dt_output)
print('Final time (s):', data1.finalTime)
print('Number of cells:', data1.NoC)
print('Number of links:', data1.NoL)
print('Water depths matrix shape:', data1.h.shape)
print('Velocity/Flow matrix shape:', data1.Q.shape)
try:
    print(data1.ssc.shape)    #can't do this because .ssc is None until we run the Sed. Transp. model
except:
    print('ssc atributte:', data1.ssc)

When **hydrodynamic_model(** *args* **)** is called with the argument *return_outcome* set as True, it creates an instance of the **highFreqData** class (which is also in egmlib). This class serves to store not only the hydrodynamic data (depths 'h', velocities 'v' and discharge 'Q'), but also its metadata (timestep, finaltime, number of cells and links, etc.), and even the outcomes from the sediment transport model if we run it later.


All records are stored in matrices with shape (*N*,*M*) where *N* is the number of records in the output series and *M* is the number of elements in the domain (#cells for *h* and *ssc*; #links for *v* and *Q*). It is important to note that the first record is not at *t=0* but at *t=*$\Delta$*t*. Also, *h* is the instant value at the timestep *t*, while all other variables are the average value along the timestep (i.e. between *t-*$\Delta$*t* and *t*)


These time series where read from txt files created by the Hydrodynamic model. The name of such files were built using the prefix and sufix defined in *fn_['h'], fn_['v'], fn_['Q']* for depths, velocities and discharges output files, respectively. The Hydrodynamic run ID comes from the first line in *anpanta.dat*.


***
    The space in disk required for the Hydrodynamic output files generated is about 0.024088794 kB per element per         time step. Therefore, for 1000 cells/links and 500 time steps, we would expect a filesize of 12,044 kB. But       remember that are 3 output files, and the number of links is nearly double the number of cells.
    
    This might be a problem for wider/longer simulations
***


## STEP 3: Calling the Hydrodynamic from the EGM simulation
Now we will create an instance of **EGMframework** (not EGMProject) that will handle all resources necessary to run the EGM Framework. Although we will just use it to update the initial condition file and run the model Hydrodynamic again.


### 3.1 Initialising an EGMframework instance
The EGMFramework class was built to manage a full EGM simulation, therefore to initialise it properly we will need to provide some information (this will be more detailed in tutorials 3 and 4). One essential info is the directory's path where the model will run, which needs to be the same place where we run the Hydrodynamic model. Any other information should be stored in a `CSV` or `JSON` file, and the name of such file should then be provided to the EGMframework.initialize() function.


Let's create a Python dictionary with one single attribute and dump it on a JSON file. Then, we'll use this file to create a new instance of EGMframework

In [ ]:
# Creating a dictionary with simulation title although it is not obrigatory either usefull (just for reference)
info = {
    'title': 'Tutorial 2'
}

# Store the dictionary in a JSON file
filename = os.path.join(run_folder, 'egm_setup.json')
with open(filename, 'w') as file:
    json.dump(info,file)

# Creating an instance of EGMFramework class
egm = EGMframework()

#  Initialise 'egm' passing the json file name and the path to the run's directory
egm.initialize(filename, run_folder)

# Print a few attributes to check if it was properly initialized
print('\nNumber of cells and links:', egm.NoC, egm.NoL)
print('Water depths boundary condition applied to:', egm.loc_HT)
print('Current step of EGM simulation:', egm.current)

### 3.2 Run the Hydrodynamic model from EGMframework
Now, let's update the initial condition using the water depths from the last time step in the previous simulation. Then, let's run the model and pass its results to a new highFreqData object

In [ ]:
#Update initial state with the last state from previous simulation
initial_condition_update(h0=data1.h[-1,:])

data2 = egm.update_hydrodynamic(returnType='highFreqData')

dif = data1.h - data2.h
bias = np.mean(np.abs(dif))
print('Average difference:', round(bias*1000,3), 'mm')
print('Lowest, Median, Highest differences =', np.min(dif)*1000, 'mm,',
      np.median(dif)*1000, 'mm,', np.max(dif)*1000, 'mm.')

# STEP 4: Analysing hydrodynamic results
In this step we will compare the results from both simulations under different aspects

## 4.1 Cell's time-series
Let's take the water levels in three different cells of the model and compare the time series from both runs. We know from Tutorial 1, that, for this domain, the *x-coordinate* increases as we move far from the tide-creek, while the *y-coordinate* increases as we move away from the inner channel. Also, there is an embankment at *x=400 m*. Thus, let compare three different points:
1. middle of lower half, near the inner channel
2. middle of lower half, far from the inner channel
3. at the upstream half, near the culvert and at the channel

In [ ]:
#A cell in the middle of lower half (x ~ 200), near the inner channel (y ~ 15)
c1 = np.where((egm.x > 190) & (egm.x <= 200) & (egm.y > 2.5) & (egm.y <= 15))[0][0]
print('Point 1: cell', egm.number[c1])

#A cell in the middle of lower half (x ~ 200), far from the inner channel (y ~ 115)
c2 = np.where((egm.x > 190) & (egm.x <= 200) & (egm.y > 105) & (egm.y <= 115))[0][0]
print('Point 2: cell', egm.number[c2])

#A cell near the culvert (x ~ 400+10) and (y ~ 15)
c3 = np.where((egm.x > 410) & (egm.x <= 420) & (egm.y > 2.5) & (egm.y <= 15))[0][0]
print('Point 3: cell', egm.number[c3])

#Time steps in hours
hours = np.arange(1, data2.NoTS+1) * data2.Dt_output / 3600

#Plotting time series
fig, ax = pyplot.subplots(ncols=3, dpi=100, figsize=(12,4), constrained_layout=True)
ax[0].plot(hours, data1.h[:,c1], label='Sim 1')
ax[0].plot(hours, data2.h[:,c1], label='Sim 2')
ax[0].set_title(str('x = %i, y = %i' % (int(egm.x[c1]), int(egm.y[c1]))))
ax[1].plot(hours, data1.h[:,c2], label='Sim 1')
ax[1].plot(hours, data2.h[:,c2], label='Sim 2')
ax[1].set_title(str('x = %i, y = %i' % (int(egm.x[c2]), int(egm.y[c2]))))
ax[2].plot(hours, data1.h[:,c3], label='Sim 1')
ax[2].plot(hours, data2.h[:,c3], label='Sim 2')
ax[2].set_title(str('x = %i, y = %i' % (int(egm.x[c3]), int(egm.y[c3]))))
for x in ax.flatten():
    x.set_xlabel('hours')
    x.set_ylabel('depths (m)')
    x.grid()
ax[2].legend(loc='best')
pyplot.show()

### 4.1.1 Cell's average velocity
The trick to plot the average velocity is to mind the signal convection between links. For all pair of linked cells, the velocity/discharge is positive when the flow occurs from cell1(inlet) to cell2(outlet). To compute the average in the cell, we will compute first the horizontal and vertical components, and then find its magnitude.


For such procedure we will use the attributes *.same_y* and *.same_x* that where created when the instance of EGMframework was initialised. These attributes store for each cell, which linked cells are in the same row and in the same column.

In [ ]:
#average water velocity [m/s]
u = np.zeros((data2.NoTS, egm.NoC), dtype=float)

for t in range(data2.NoTS):
    for c in range(egm.NoC):            
        # Magnitude of cell's velocity vector
        vel_x = np.average(data2.v[t,egm.same_y[c]])
        vel_y = np.average(data2.v[t,egm.same_x[c]])
        u[t,c] = (vel_x**2 + vel_y**2)**0.5

#now let's plot the average velocity over time on each selected cell (for the second simulation only)
fig, ax = pyplot.subplots(dpi=100, figsize=(8,6), constrained_layout=True)
ax.plot(hours, u[:,c1], label=str('x = %i, y = %i' % (int(egm.x[c1]), int(egm.y[c1]))))
ax.plot(hours, u[:,c2], label=str('x = %i, y = %i' % (int(egm.x[c2]), int(egm.y[c2]))))
ax.plot(hours, u[:,c3], label=str('x = %i, y = %i' % (int(egm.x[c3]), int(egm.y[c3]))))
ax.set_xlabel('hours')
ax.set_ylabel('average velocity (m/s)')
ax.grid()
ax.legend(loc='best')
pyplot.show()

## 4.2 Water depths profile
Now, let's select three rows of the domain and create different plots at different parts of the tidal cycle

In [ ]:
#Profile at the inner channel's margin (y = 10m)
p1 = np.where((egm.y == 10) & (egm.x >= 0))[0]

# Profile at short distance from inner channel (y = 50m)
p2 = np.where((egm.y == 50) & (egm.x >= 0))[0]

# Profile distant from inner channel (y = 100m)
p3 = np.where((egm.y == 100) & (egm.x >= 0))[0]

# Time stamps to plot
t = np.where((hours == 2) | (hours == 4) | (hours == 6) | (hours == 10))[0]
x = egm.x.flatten()

#Plotting water depths profiles at different time
fig, ax = pyplot.subplots(nrows=4, dpi=100, figsize=(8,12), constrained_layout=True)
for i in range(4):
    ax[i].plot(egm.x[p1], data2.h[t[i],p1], label='y =  10 m')
    ax[i].plot(egm.x[p2], data2.h[t[i],p2], label='y =  50 m')
    ax[i].plot(egm.x[p3], data2.h[t[i],p3], label='y = 100 m')
    ax[i].plot([400., 400.], [0., 0.4], color='black', linewidth=3, label='Embankment')
    ax[i].text(420, 0.4, str('Hour = %i' % int(hours[t[i]])), va='top', ha='left')
    ax[i].grid()
    ax[i].set_xlabel('distance from tide creek (m)')
    ax[i].set_ylabel('water depth (m)')
ax[0].legend(loc="best")
pyplot.show()

## 4.3 Water levels and velocity/discharge animation
Animations are very good to see throughout the time what is going on in the entire profile. The code below will create an animation with water levels (depth + bottom elevation) at two different rows of the domain. It also include the bathtub profile. 

In [ ]:
%matplotlib inline
import matplotlib.animation as mplanim
from IPython.display import HTML

#Profile at the inner channel's margin (y = 10m)
p4 = np.where((egm.y == 10) & (egm.x >= 0) & (egm.x < 600))[0]

# Profile distant from inner channel (y = 100m)
p5 = np.where((egm.y == 100) & (egm.x >= 0) & (egm.x < 600))[0]

# Bathtub simulation
zTide = data2.h[:,0]+egm.z0[0]
zbtub = np.zeros(data2.h.shape)
for i in range(int(data2.NoTS)):
    for j in range(egm.NoC):
        zbtub[i,j] = max(zTide[i], egm.z0[j])

# Create figure and plots with a first draw
fig, ax = pyplot.subplots(dpi=100, figsize=(6,3), constrained_layout=True)
near, = ax.plot(egm.x[p4], data2.h[0,p4] + egm.z0[p4], label='y =  10 m')
far,  = ax.plot(egm.x[p5], data2.h[0,p5] + egm.z0[p5], label='y = 100 m')
btub, = ax.plot(egm.x[p5], egm.z0[p5], color='black', label='bathtub')
time_text = ax.text(0.5, 0.05, str('%5.2f hours' % hours[0]), transform=ax.transAxes, va='center', ha='center')

# Indexes of time steps to update the image
t12 = np.where(hours == 12)[0][0]
steps = np.arange(t12)

def update_image(t):
    ''' Update y-data of plot given the time's index, t '''
    near.set_ydata(data2.h[t,p4] + egm.z0[p4])
    far.set_ydata(data2.h[t,p5] + egm.z0[p5])
    btub.set_ydata(zbtub[t,p5])
    time_text.set_text(str('%5.2f hours' % hours[t]))

def init():
    ''' Initialise the figure '''    
    ax.set_ylim(-0.1, 0.6)
    ax.set_xlim(-10, 610)
    ax.set_xlabel('Distance from tide creek (m)')
    ax.set_ylabel('Elevation (m)')
    ax.grid()
    ax.legend(loc='center right')

anim = mplanim.FuncAnimation(fig, update_image, steps, init_func=init, interval=100)#, blit=True) #, repeat=False)
HTML(anim.to_html5_video())

The velocity is given at the links between cells, which makes the plotting of this variable a bit more complex. Thus, to make a similar animation, instead of provide the cells' number along the same row, we must find the links' number. We also need to mind that the embankment is actually a gap in the sequence of links or, more precisely, a missed linked between the cells adjacent to the structure.


Once we build up the sequence of links in the each part of the row, the procedure to make the animation is pretty much the same. If you want you can review the code below to plot the discharge by changing all *data2.v* by *data2.Q*

In [ ]:
# Finding links along profile p4 (remember that there is an embankment in the middle)
links4, x4 = [[],[]], [[],[]]
sub = 0
for c2 in p4:
    c1 = c2 - 1
    try:
        l = np.where((egm.cell1 == c1) & (egm.cell2 == c2))[0][0]
        links4[sub].append(l)
        x4[sub].append(0.5 * (egm.x[c1] + egm.x[c2]))
    except:
        sub += 1
links4 = np.array(links4)
x4 = np.array(x4)

# Finding links along profile p5
links5, x5 = [[],[]], [[],[]]
sub = 0
for c2 in p5:
    c1 = c2 - 1
    try:
        l = np.where((egm.cell1 == c1) & (egm.cell2 == c2))[0][0]
        links5[sub].append(l)
        x5[sub].append(0.5 * (egm.x[c1] + egm.x[c2]))
    except:
        sub += 1
links5 = np.array(links5)
x5 = np.array(x5)

# Create figure and plots for a first draw
fig, ax = pyplot.subplots(dpi=100, figsize=(6,3), constrained_layout=True)
near_ds, = ax.plot(x4[0], data2.v[0,links4[0]], label='y =  10 m')
near_us, = ax.plot(x4[1], data2.v[0,links4[1]], color='tab:blue')
far_ds,  = ax.plot(x5[0], data2.v[0,links5[0]], label='y = 100 m')
far_us,  = ax.plot(x5[1], data2.v[0,links5[1]], color='tab:orange')
time_text = ax.text(0.5, 0.05, str('%5.2f hours' % hours[0]), transform=ax.transAxes, va='center', ha='center')

def update_image2(t):
    ''' Update y-data of plot given the time's index, t '''
    near_ds.set_ydata(data2.v[t,links4[0]])
    near_us.set_ydata(data2.v[t,links4[1]])
    far_ds.set_ydata(data2.v[t,links5[0]])
    far_us.set_ydata(data2.v[t,links5[1]])
    time_text.set_text(str('%5.2f hours' % hours[t]))

def init2():
    ''' Initialise the figure '''    
    ax.set_ylim(-0.06, 0.06)
    ax.set_xlim(-10, 610)
    ax.set_xlabel('Distance from tide creek (m)')
    ax.set_ylabel('Velocity (m/s)')
    ax.grid()
    ax.legend(loc='upper right')

anim = mplanim.FuncAnimation(fig, update_image2, steps, init_func=init2, interval=100)
HTML(anim.to_html5_video())

# The End
In this tutorial you saw how to run the Hydrodynamic model straight way, once all input's files are placed in the same location.

We also include a run from the EGMframework class, as it is the way we will use later on real EGM simulations.

Last, some few plotting exercices were carried on to show how data is stored and how it correlates with domain properties.